# Algoritma perhitungan potensi ekspansi di suatu kota
### Mengfilter raw data
1. Menghapus baris yang tidak memiliki alamat
2. Mengfilter dari kategori
3. Mengfilter berdasarkan nama tempatnya

### Membuat cluster daerah yang berpotensi dalam satu kota
1. menggunakan algoritma k-means pada titik titik pasar untuk mengcluster lokasi yang berdekatan
2. Menentukan jumlah cluster yang terbaik dengan <i>elbow method</i>
3. Mengambil titik tengah dari masing masing cluster
4. Daerah titik tengah merupakan daerah potensial untuk mencari calon MITRA


### Menampilkan titik pasar, perumahan, apartemen dan pusat circle pada peta 
1. mengfilter pasar, perumahan dan apartemen yang berada dalam radius 3 km relatif terhadap pusat cluster
2. menggunakan library folium untuk menvisualisasi data
3. data yang divisualisasikan adalah titik pasar, perumahan, dan apartemen yang terletak dalam suatu cluster

### Menghitung potensi dari kota tersebut berdasarkan bobot indeks yang diterapkan
1. indeks pertama 
- dengan pertimbangan perumahan memiliki jumlah anggota keluarga yang relatif lebih banyak dibandingkan apartemen, maka 
** indeks perumahan = 2
** indeks apartemen = 1
2. indeks kedua
- dengan pertimbangan jarak titik tersebut dengan pasar mempengaruhi keinginan seseorang untuk memesan dari aplikasi TSco, maka
** indeks titik dengan lokasi > 1,5 km dari pasar terdekat = 2
** indeks titik dengan lokasi <= 1,5 km dari pasar terdekat = 1

In [33]:
import folium 
import pandas as pd
import math 
import numpy as np

In [64]:
kota = str(input("Masukkan nama kota : "))
koor_kota = {"aceh" : [4.0432597,94.4042103], "medan" : [3.6426183,98.5294062], "pekanbaru" : [0.5139625,101.3711353], "jambi": [-1.6102634,103.5394878], "padang":[-0.9342375,100.2511826]}
tabel = pd.read_csv("Data/jarak/{}_jarakFIX.csv".format(kota)).drop(columns="Unnamed: 0")
pasar = pd.read_csv("Data/bersih/{}_pasar_bersih.csv".format(kota)).drop(columns=["Unnamed: 0", "Subtitle"])
perumahan = pd.read_csv("Data/bersih/{}_perumahan_bersih.csv".format(kota)).drop(columns=["Unnamed: 0", "Subtitle"])
apartemen = pd.read_csv("Data/bersih/{}_apartemen_bersih.csv".format(kota)).drop(columns=["Unnamed: 0", "Subtitle"])

Masukkan nama kota : padang


In [65]:
tabel

,Nama Cluster,Latitude,Longitude,Jarak TC ke pasar 1 (km),Jarak TC ke pasar 2 (km),Jarak TC ke pasar 3 (km),Jarak TC ke pasar 4 (km),Jarak TC ke pasar 5 (km),Jarak TC ke pasar 6 (km),Jarak TC ke pasar 7 (km),...,Jarak TC ke perumahan 203 (km),Jarak TC ke perumahan 204 (km),Jarak TC ke perumahan 205 (km),Jarak TC ke perumahan 206 (km),Jarak TC ke perumahan 207 (km),Jarak TC ke perumahan 208 (km),Jarak TC ke perumahan 209 (km),Jarak TC ke perumahan 210 (km),Jarak TC ke perumahan 211 (km),Jarak TC ke perumahan 212 (km)
0,Cluster ke 1,-0.929398,100.354298,0.347011,0.347011,0.927056,8.008793,5.849434,0.372620,2.265623,...,342.190680,341.076567,902.151358,2213.234349,1853.913325,1853.219668,2212.087929,1852.894980,532.226263,1853.575757
1,Cluster ke 2,-0.874572,100.345312,6.019143,6.019143,6.981517,8.577793,8.610812,5.989312,8.411124,...,336.013040,334.898912,904.622575,2214.869181,1857.677781,1856.978155,2213.720703,1856.651572,526.174007,1857.333001
2,Cluster ke 3,-0.926309,100.397140,5.095771,5.095771,5.258606,4.006699,1.129154,5.111205,4.669195,...,342.643991,341.536344,897.616445,2208.532808,1849.877880,1849.181109,2207.386098,1848.855434,533.616590,1849.536551
3,Cluster ke 4,-0.950396,100.363541,2.799425,2.799425,2.026089,8.507616,5.707306,2.832122,0.314529,...,344.660071,343.546810,900.588384,2211.970792,1851.906053,1851.214319,2210.825140,1850.890242,534.773855,1851.570808
4,Cluster ke 5,-0.898106,100.369996,3.951966,3.951966,4.817950,5.441917,4.852343,3.936816,5.679967,...,339.041561,337.930553,901.313818,2211.862870,1854.020843,1853.322442,2210.715189,1852.996249,529.601292,1853.677543
5,Cluster ke 6,-0.848812,100.370063,9.087218,9.087218,10.040422,7.679991,9.102314,9.063636,11.122574,...,333.638946,332.529227,902.679203,2212.440126,1856.617323,1855.913044,2211.290566,1855.584985,524.494475,1856.266918
6,Cluster ke 7,-0.828891,100.327447,11.375979,11.375979,12.315253,12.723171,13.588171,11.344711,13.824203,...,330.683836,329.568125,907.824347,2217.390114,1861.838710,1861.134799,2216.239973,1860.806855,520.720038,1861.488750
7,Cluster ke 8,-0.899608,100.407834,7.046968,7.046968,7.561615,1.254473,2.512867,7.049085,7.489640,...,339.929731,338.824448,897.197010,2207.661885,1850.237870,1849.537217,2206.514109,1849.210311,531.286537,1849.891851


In [66]:
pasar

,Title,Category,Address,Latitude,Longitude
0,Pasar Pagi,Shopping mall,Jln Teri Pasar Pagi Kelurahan. Rimbo Kaluang. ...,-0.928366,100.351353
1,Pasar Pagi,Shopping mall,Jln Teri Pasar Pagi Kelurahan. Rimbo Kaluang. ...,-0.928366,100.351353
2,Pasar Ikan Pantai Padang,Market,Jl. Samudera. Purus. Kec. Padang Bar.. Kota Pa...,-0.937092,100.351088
3,Pasar Belimbing Kuranji Kota Padang,Shopping mall,Jl. Pepaya Raya. Kuranji. Kec. Kuranji. Kota P...,-0.897617,100.418940
4,Pasar Murah,Market,Jl. Korong Gadang. Korong Gadang. Kec. Kuranji...,-0.922161,100.406410
5,Pasar Pagi Juanda,Market,39C2+QF4. Rimbo Kaluang. Kec. Padang Bar.. Kot...,-0.928113,100.351203
6,Pasar Raya Blok II (Inpres II),Market,Jl. Ps. Baru. Kp. Jao. Kec. Padang Bar.. Kota ...,-0.948470,100.361468
7,Blok IV Pasaraya Padang,Produce market,Jl. Ps. Baru. Kp. Jao. Kec. Padang Bar.. Kota ...,-0.948470,100.361468
8,Pasar Raya Kota Padang,Shopping mall,Unnamed Road. Kp. Pd.. Kec. Padang Bar.. Kota ...,-0.957142,100.364695
9,Pasar Siteba,Market,Jl. Raya Siteba No.18. Surau Gadang. Kec. Nang...,-0.899064,100.370765


In [67]:
perumahan

,Title,Category,Address,Latitude,Longitude
0,Perumahan Pondok Ranah Minang,Gated community,Koto Lalang. Kec. Lubuk Kilangan. Kota Padang....,-0.963489,100.431227
1,perumahan parak karakah regency,Gated community,Unnamed Road. Kubu Dalam Parak Karakah. Kec. P...,-0.946094,100.388587
2,Komplek Talago Permai,Gated community,Komplek Talago Permai. Alai Parak Kopi. Kec. P...,-0.929377,100.373200
3,Perumahan Parak Karakah Asri,Gated community,Perumahan. Kubu Dalam Parak Karakah. Asri. Kot...,-0.945043,100.392164
4,Perumahan Minang Regency,Gated community,Jl. Korong Gadang No.33. Korong Gadang. Kec. K...,-0.922284,100.407899
...,...,...,...,...,...
207,Perum Padang Indah,Gated community,Jl. Padang Indah. Padangsambian Klod. Kec. Den...,-8.660336,115.182830
208,Perumahan Padang Alipan,Housing authority,4547+C58. Jaya. Telluwanua. Kota Palopo. Sulaw...,-2.893963,120.162953
209,Perumahan Nusa Padang Estate II,Gated community,Jl. Taman Sekar. Padangsambian. Kec. Denpasar ...,-8.653341,115.183099
210,Perumahan Citra Garden,Housing development,Perumahan. Jl. Citra Garden No.9. Padang Bulan...,3.544690,98.652715


In [68]:
apartemen

,Title,Category,Address,Latitude,Longitude
0,Apartemen77,Apartment complex,Unnamed Road. Parak Laweh Pulau Air Nan XX. Ke...,-0.974375,100.392119


# Mencari index dari data dengan jarak dibawah 3 km

In [69]:
cluster_pasar = []
for i in range(len(tabel)):
    cluster_pasar.append([])
    
for c in range(len(tabel)):
    for d in range(len(pasar)):
        if tabel["Jarak TC ke pasar {} (km)".format(d+1)][c] <= 3:
            cluster_pasar[c].append(d)

cluster_apartemen = []
for i in range(len(tabel)):
    cluster_apartemen.append([])
    
for x in range(len(tabel)):
    for y in range(len(apartemen)):
        if tabel["Jarak TC ke apartemen {} (km)".format(y+1)][x] <= 3:
            cluster_apartemen[x].append(y)

cluster_perumahan = []
for i in range(len(tabel)):
    cluster_perumahan.append([])
    
for a in range(len(tabel)):
    for b in range(len(perumahan)):
        if tabel["Jarak TC ke perumahan {} (km)".format(b+1)][a] <= 3:
            cluster_perumahan[a].append(b)


In [70]:
cluster_apartemen

[[], [], [], [], [], [], [], []]

# mendapatkan koordinat Titik tengah Claster sebagai pusat claster

In [71]:
koor_TC = []

for i in range(len(tabel)):
    lat = tabel["Latitude"].loc[i]
    lng = tabel["Longitude"].loc[i]
    koor_TC.append([lat, lng])
print(koor_TC)


[[-0.929398, 100.35429818333334], [-0.8745723666666667, 100.34531203333331], [-0.9263093, 100.3971399], [-0.950395675, 100.3635406875], [-0.898106, 100.3699964], [-0.8488125, 100.3700625], [-0.8288907, 100.3274469], [-0.89960765, 100.4078342]]


# Mendapatkan koordinat dari pasar, perumahan dan apartemen

In [72]:
circle_koor_pasar = []
circle_koor_apartemen = []
circle_koor_perumahan = []

for i in range(len(tabel)):
    circle_koor_apartemen.append([])
    circle_koor_perumahan.append([])
    circle_koor_pasar.append([])

for a in range(len(cluster_pasar)):
    for b in range(len(cluster_pasar[a])):
        lat = pasar["Latitude"][cluster_pasar[a]][cluster_pasar[a][b]]
        lng = pasar["Longitude"][cluster_pasar[a]][cluster_pasar[a][b]]
        circle_koor_pasar[a].append([lat, lng])
    
for c in range(len(cluster_apartemen)):
    for d in range(len(cluster_apartemen[c])):
        lat = apartemen["Latitude"][cluster_apartemen[c]][cluster_apartemen[c][d]]
        lng = apartemen["Longitude"][cluster_apartemen[c]][cluster_apartemen[c][d]]
        circle_koor_apartemen[c].append([lat, lng])

for e in range(len(cluster_perumahan)):
    for f in range(len(cluster_perumahan[e])):
        lat = perumahan["Latitude"][cluster_perumahan[e]][cluster_perumahan[e][f]]
        lng = perumahan["Longitude"][cluster_perumahan[e]][cluster_perumahan[e][f]]
        circle_koor_perumahan[e].append([lat, lng])


In [73]:
m = folium.Map(location=koor_kota[kota], zoom_start = 13) 

for i in range(len(circle_koor_apartemen)):
    for j in range(len(circle_koor_apartemen[i])):
        folium.Marker(circle_koor_apartemen[i][j], icon = folium.Icon(icon = "glyphicon-shopping-cart", color = "red")).add_to(m)

for k in range(len(circle_koor_perumahan)):
    for l in range(len(circle_koor_perumahan[k])):
        folium.Marker(circle_koor_perumahan[k][l], icon = folium.Icon(icon = "home", color = "blue")).add_to(m)

for x in range(len(circle_koor_pasar)):
    for y in range(len(circle_koor_pasar[x])):
        folium.Marker(circle_koor_pasar[x][y], icon = folium.Icon(icon = "home", color = "black")).add_to(m)

for n in range(len(koor_TC)):
    folium.Marker(koor_TC[n], icon = folium.Icon(icon = "home", color = "green")).add_to(m)
    folium.Circle(
        radius=3000,
        location=koor_TC[n],
        popup="The Waterfront",
        color="crimson",
        fill=False,
    ).add_to(m)
m

In [74]:
pasar_u = []
pr_u = []
ap_u = []

def unik(cluster, l):
    for i in range(len(cluster)):
        for j in range(len(cluster[i])):
            l.append(cluster[i][j])
    return list(set(l))

pasar_unik = unik(cluster_pasar, pasar_u)
pr_unik = unik(cluster_perumahan, pr_u)
ap_unik = unik(cluster_apartemen, ap_u)


In [75]:
def tbaru(l1, l2):
    ind = []
    for i in range(len(l1)):
        ind.append(i)

    x = l2[["Title", "Latitude", "Longitude"]].loc[l1]
    x.index = ind
    return x
t_pasar = tbaru(pasar_unik, pasar)
t_perumahan = tbaru(pr_unik, perumahan)
t_apar = tbaru(ap_unik, apartemen)


In [76]:
t_pasar.to_csv("{}_t_pasar.csv".format(kota))

In [77]:
t_perumahan.to_csv("{}_t_perumahan.csv".format(kota))

In [78]:
t_apar.to_csv("{}_t_apar.csv".format(kota))